<a href="https://colab.research.google.com/github/sergfer26/Proyectos-II/blob/main/colab/colabServer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conectarse a Google Colab usando SSH 

## 1. Instalar SSH en Windows

Para conectarnos desde Windows vía SSH necesitamos instalar un cliente de SSH. La forma más simple de instalar SSH se describe en [este artículo de Microsoft](https://docs.microsoft.com/en-us/windows-server/administration/openssh/openssh_install_firstuse).

## 2. Ngrok

Para conectarnos al servidor de Colab con SSH necesitamos tunelizar la conexión del servidor de Colab con el cliente de _Ngrok_. Para ello, primero cree una cuenta en [el sitio de Ngrok](https://dashboard.ngrok.com/signup).

## 3. Reenvío de puerto SSH desde Colab

El siguiente código genera una contraseña aleatoria para el usuario `root` en el servidor de Colab. Utilizaremos esta información más adelante.

In [ ]:
#1 - Setup ssh/user 

#Generate a random root password
import random, string
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(30))


#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config

print("username: root")
print("password: ", password)

#Run sshd
from IPython import get_ipython
get_ipython().system_raw('/usr/sbin/sshd -D &')

A continuación descargamos el cliente de _Ngrok_ al servidor de Colab.

In [ ]:
# 2 - Download Ngrok

! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

El siguiente paso es tunelizar el puerto 22 del servidor de Colab a través de _Ngrok_. Debemos ir a [https://dashboard.ngrok.com/auth](https://dashboard.ngrok.com/auth) para obtener nuestro `authtoken`. 

Cuando ejecutemos la siguiente casilla de código, debemos copiar el `authtoken` y pegarlo en el _prompt_ que aparecerá a continuación.

_Nota_: El `authtoken` generado por _Ngrok_ nos permite autenticarnos, de tal modo que podamos ver los detalles de la conexión que está siendo tunelizada desde [el _dashboard_ de _Ngrok_](https://dashboard.ngrok.com/endpoints/status).

In [ ]:
# 3 - Setup Ngrok - authtoken

#Ask token
print("Get your authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

## 4. Establecer conexión SSH

Después del anterior paso, ve a [https://dashboard.ngrok.com/status](https://dashboard.ngrok.com/status) y deberíamos poder ver algo así:

>URL: `tcp://0.tcp.ngrok.io:[ngrok_port]`
>
>Client IP: \*\*\*.\*\*\*.\*\*\*.\*\*\*
>
>Region: `us`
>
>Established: May 17th, 2019 08:44:18 UTC


Finalmente, podemos establecer la conexión con SSH:

```bash
ssh root@0.tcp.ngrok.io -p [ngrok_port]

> Igresa la contraseña creada anteriormente

```

## Desarrollo Remoto usando SSH

Para conectarnos al servidor de Colab y hacer desarrollo desde ahí con nuestro cliente de Visual Studio Code localmente podemos seguir las instrucciones en [https://code.visualstudio.com/docs/remote/ssh](https://code.visualstudio.com/docs/remote/ssh).

## Transferir archivos al servidor Colab

Instalen WinSCP [https://winscp.net/download/WinSCP-5.19.4-Setup.exe](https://winscp.net/download/WinSCP-5.19.4-Setup.exe) e inicien la conexión con las credenciales:

> Host name: `0.tcp.ngrok.io`
> 
> Port number: \<Puerto establecido por Ngrok\>
> 
> User name: root
> 
> Password: \<Contraseña antes generada\>


Una vez conectados al servidor pueden abrir una sesion de screen y ahi correr su codigo.

* Screen

Les ha pasado una situación en la que tienen una tarea de larga duración en una máquina remota y, de repente, la conexión se interrumpe, la sesión SSH finaliza y el trabajo se pierde? 

Por suerte, existe un programa llamado screen que permite retomar las sesiones.

Los procesos que se ejecutan con screen continuarán ejecutándose aun cuando ls ventana no sea visible o incluso si se desconecta.

* Uso básico de la pantalla de Linux

Los pasos más básicos para comenzar con la pantalla:

1. En el shell del sistema, escribir el comando screen.
2. Ejecuten el programa deseado.
3. Utiliza la secuencia de teclas Ctrl-a + Ctrl-d para desconectarse de la sesión de pantalla.
4. Vuelvan a conectarse a la sesión de pantalla escribiendo screen -r.

El comando screen abrirá una sesión de pantalla, creará una nueva ventana e iniciará un shell en esa ventana.

Se puede obtener una lista de comandos escribiendo:

Ctrl + a?

Cuando inicia una nueva sesión de pantalla, crea una única ventana con un caparazón en ella.

Algunos de los comandos más comunes para administrar Screen:

Ctrl + a c Crea una nueva ventana (con shell).
Ctrl + a " Lista todas las ventanas.
Ctrl + a 0 Cambiar a la ventana 0 (por número).
Ctrl + a A Cambiar el nombre de la ventana actual.
Ctrl + a S Divide la región actual horizontalmente en dos regiones.
Ctrl + a | Divide la región actual verticalmente en dos regiones.
Ctrl + una pestaña Cambia el foco de entrada a la siguiente región.
Ctrl + a Ctrl + a Alternar entre la ventana actual y la anterior
Ctrl + a Q Cerrar todas las regiones menos la actual.
Ctrl + a X Cierra la región actual.

* Desconectar de la sesión de pantalla de Linux
Pueden desconectarse de la sesión de pantalla escribiendo:

Ctrl + a d

El programa que se ejecuta en la sesión de pantalla continuará ejecutándose después de que se desconecten de la sesión.

* Volver a conectar a una screen de Linux
Para reanudar su sesión de screen:

screen -r

En caso de que tener varias sesiones de pantalla ejecutándose, deberán agregar el ID de sesión de pantalla después del parametro r.

Para encontrar el ID de sesión, enumera las sesiones de pantalla en ejecución actuales con:

pantalla -ls

Ejemplo de salida:
Hay screen en:
    10835.pts-0.linuxize-desktop (independiente)
    10366.pts-0.linuxize-desktop (independiente)

Para restaurar la pantalla 10835.pts-0:

screen -r 10835